In [364]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import math

# Preprocessing

In [365]:
raw_path = '../data/raw'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [366]:
train

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,...,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,...,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,...,06:00:00,0,0,0,54-103-006-01,0,0,0,0,0
951,54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:15:00,0,0,0,54-701-039-01,0,0,0,0,0
952,54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,...,08:00:00,1,0,0,54-002-014-01,1,0,1,0,0
953,54-501-022-01,Ж,в браке в настоящее время,европейская,Русские,Атеист / агностик,5 - ВУЗ,"представители законодат. органов власти,...",1,0,...,08:30:00,0,1,0,54-501-022-01,0,0,0,0,0


In [367]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког']

In [368]:
def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df

In [369]:
def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df

In [370]:
def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
    df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
    df = df.drop(columns=[EDU_COL])
    return df

In [371]:
def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[SEX_COL].value_counts().index[0]
    df[SEX_COL] = df[SEX_COL].fillna(most_freq)
    return df

In [372]:
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')

    ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
    return df
    

In [373]:
def delete_nan_value(df: pd.DataFrame) ->pd.DataFrame:
    ##Smoking
    period_smoke = list(df['Возраст курения'])
    number_of_сigarettes = list(df['Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        if row['Статус Курения'] == 'Никогда не курил(а)':
            if pd.isna(row['Возраст курения']):
                period_smoke[count] = 0
            if pd.isna(row['Сигарет в день']):
                number_of_сigarettes[count] = 0
        count += 1
    df = df.drop(columns=['Возраст курения', 'Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст курения'] = period_smoke[count]
        df.loc[index,'Сигарет в день'] = number_of_сigarettes[count]
        count += 1 
    df = df[(np.isnan(df['Возраст курения'])) == False] 
    df['Возраст курения'] = df['Возраст курения'].astype(np.int8)
    df = df[(np.isnan(df['Сигарет в день'])) == False] 
    df['Сигарет в день'] = df['Сигарет в день'].astype(np.int8)
    

    ## Пассивное курение 
    frequency_second_hand_smoke = list(df['Частота пасс кур'])
    count = 0
    for index, row in df.iterrows():
        if row['Пассивное курение'] == 0:
            if pd.isna(row['Частота пасс кур']):
                frequency_second_hand_smoke[count] = 'менее 1 раза в день'
        count += 1
    df = df.drop(columns=['Частота пасс кур'])
    df.insert(21, 'Частота пасс кур', frequency_second_hand_smoke, True)
    df = df[(pd.isna(df['Частота пасс кур'])) == False] 
    df['Частота пасс кур'] = df['Частота пасс кур'].astype('category')
    
    ## алкоголь
    drinking_period = list(df['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        if row['Алкоголь'] == 'никогда не употреблял':
            if pd.isna(row['Возраст алког']):
                drinking_period[count] = 0
        count += 1
    df = df.drop(columns=['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст алког'] = drinking_period[count]
        count += 1
    df = df[(pd.isna(df['Возраст алког'])) == False] 
    df['Возраст алког'] = df['Возраст алког'].astype(np.int8)
    return df

In [374]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = cast_types(df)
    df = add_ord_edu(df)
    df = delete_nan_value(df)
    return df

In [375]:
def preprocess_target(df: pd.DataFrame) -> pd.DataFrame:
    df[TARGET_COLS] = df[TARGET_COLS].astype(np.int8)
    return df

def extract_target(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df, target = df.drop(TARGET_COLS, axis=1), df[TARGET_COLS]
    return df, target

In [376]:
import pickle
from typing import Union
from pandas import DataFrame
from pandas.core.indexes.base import Index as PandasIndex


def save_as_pickle(obj: Union[DataFrame, PandasIndex], path: str) -> None:
    if isinstance(obj, DataFrame):
        obj.to_pickle(path)
    elif isinstance(obj, PandasIndex):
        with open('path', 'wb') as f:
            pickle.dump(obj, f)

In [377]:
train = preprocess(train)
test = preprocess(test)


##только для train()
df, target = extract_target(train)
target = preprocess_target(target)
save_as_pickle(target, '../data/processed/target_for_data_for_train.pkl')
save_as_pickle(df, '../data/interim/data_for_train.pkl')
##только для test
save_as_pickle(test, '../data/interim/test.pkl')

C:\Users\dsfs-\AppData\Local\Temp\ipykernel_14648\492307506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[TARGET_COLS] = df[TARGET_COLS].astype(np.int8)


In [378]:
target

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0
...,...,...,...,...,...
54-103-006-01,0,0,0,0,0
54-701-039-01,0,0,0,0,0
54-002-014-01,1,0,1,0,0


In [379]:
train.columns

Index(['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Профессия',
       'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни',
       'Сахарный диабет', 'Гепатит', 'Онкология',
       'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Пассивное курение', 'Частота пасс кур', 'Алкоголь',
       'Время засыпания', 'Время пробуждения', 'Сон после обеда',
       'Спорт, клубы', 'Религия, клубы', 'Артериальная гипертензия', 'ОНМК',
       'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность',
       'Прочие заболевания сердца', 'Образование_ord', 'Возраст курения',
       'Сигарет в день', 'Возраст алког'],
      dtype='object')

# Feature generation

 Из столбцов "Время пробуждения" и "Время засыпания" создадим два признака: 1. Является человек совой/Жаворонком/Голубем 2. Количество сна в часах

In [380]:
interim_data_path = '../data/interim/'
train = pd.read_pickle(os.path.join(interim_data_path, 'data_for_train.pkl'))
test = pd.read_pickle(os.path.join(interim_data_path, 'test.pkl'))

In [381]:
import datetime
def check_owl_or_lark(df: pd.DataFrame) -> pd.DataFrame:
    check_owl_or_lark = []
    for index, row in df.iterrows():
        if pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S' ) >= pd.to_datetime('23:00:00',format= '%H:%M:%S' ) or  pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S' ) < pd.to_datetime('5:00:00',format= '%H:%M:%S') :
            if pd.to_datetime(row['Время пробуждения'],format= '%H:%M:%S' ) >= pd.to_datetime('10:00:00',format= '%H:%M:%S' ):
                check_owl_or_lark.append('owl')
            else:
                check_owl_or_lark.append('pigeon')
        else:
            if pd.to_datetime(row['Время пробуждения'],format= '%H:%M:%S' ) < pd.to_datetime('10:00:00',format= '%H:%M:%S' ):
                 check_owl_or_lark.append('lark')
            else:
                check_owl_or_lark.append('pigeon')

    df.insert(25, 'Сова/Жаворонок/Голубь', check_owl_or_lark, True)
    df['Сова/Жаворонок/Голубь'] = df['Сова/Жаворонок/Голубь'].astype('category')
    return df   

In [382]:
def amount_of_sleep(df: pd.DataFrame) -> pd.DataFrame:
    amount_of_sleep = []
    for index, row in df.iterrows():
        time  = pd.to_datetime(row['Время пробуждения'], format= '%H:%M:%S') - pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S')
        amount_of_sleep.append(int(time.to_pytimedelta().seconds/3600))
    
    df.insert(26, 'Часов сна', amount_of_sleep, True)
    df['Часов сна'] = df['Часов сна'].astype(np.int8)
    return df

    

In [383]:
def remove_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns=['Время засыпания', 'Время пробуждения'],axis = 1)
    return df

In [384]:
def feature_generation(df: pd.DataFrame) -> pd.DataFrame:
    df = check_owl_or_lark(df)
    df = amount_of_sleep(df)
    df = remove_unnecessary_columns(df)
    #df = cast_types(df)
    return df

In [385]:
train = feature_generation(train)
test = feature_generation(test)


save_as_pickle(test, '../data/processed/test.pkl')
save_as_pickle(train, '../data/processed/data_for_train.pkl')


# Modeling

In [386]:
import os
import pandas as pd
import numpy as np

from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *
from sklearn.metrics import precision_score, f1_score, recall_score, roc_auc_score

#%pip install catboost
import catboost
from catboost import Pool,cv
from catboost import CatBoostClassifier

import skmultilearn
from skmultilearn.model_selection import IterativeStratification

#%pip install category_encoders
import category_encoders as ce
#%pip install lightgbm
import lightgbm as ltb

## Training

In [387]:
processed_data_path = '../data/processed/'
train = pd.read_pickle(os.path.join(processed_data_path, 'data_for_train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target_for_data_for_train.pkl'))


In [389]:
COL =['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Профессия',
       'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни',
       'Сахарный диабет', 'Гепатит', 'Онкология',
       'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Пассивное курение', 'Частота пасс кур', 'Алкоголь',
       'Сова/Жаворонок/Голубь', 'Часов сна', 'Сон после обеда', 'Спорт, клубы',
       'Религия, клубы', 'Образование_ord', 'Возраст курения',
       'Сигарет в день', 'Возраст алког']
       
COL_TARGET = ['Артериальная гипертензия', 'ОНМК',
       'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность',
       'Прочие заболевания сердца']

In [554]:
def metrics_print_for_catboost(y_predict, y_predict_proba, y_true, average):
    print(average)
    print('precision ', precision_score(y_true, y_predict, average=average))
    print('recall ', recall_score(y_true, y_predict,average=average))
    print('f1 ', f1_score(y_true, y_predict,average=average))
    if average == 'micro':
       # y_predict_proba = np.transpose([pred[:, 1] for pred in y_predict_proba])
        print('ROCAUC', roc_auc_score(y_true, y_predict_proba, average = average))
    else:
        #y_predict_proba = np.transpose([pred[:, 1] for pred in y_predict_proba])
        print('ROCAUC', roc_auc_score(y_true, y_predict_proba, average = None))
    print('\n')

def metrics_print_for_lightgbm(y_predict, y_predict_proba, y_true, average):
    print(average)
    print('precision ', precision_score(y_true, y_predict, average=average))
    print('recall ', recall_score(y_true, y_predict,average=average))
    print('f1 ', f1_score(y_true, y_predict,average=average))
    if average == 'micro':
        y_predict_proba = np.transpose([pred[:, 1] for pred in y_predict_proba])
        print('ROCAUC', roc_auc_score(y_true, y_predict_proba, average = average))
    else:
        y_predict_proba = np.transpose([pred[:, 1] for pred in y_predict_proba])
        print('ROCAUC', roc_auc_score(y_true, y_predict_proba, average = None))
    print('\n')

In [446]:
CATEGORIES_COL_AFTER_PREP = ['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь', 'Сова/Жаворонок/Голубь']

In [429]:
def turning_types(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if col in CATEGORIES_COL_AFTER_PREP:
            df[col] = df[col].astype('category')
        else:
            df[col] = df[col].astype(np.int8)
    return df

In [393]:
#Метрики несбалансированные, при рабиение на train\test мы должны учитывать соотношение 0 и 1.
#Данная функция позволяет работать со случаем multilabes
def split_data(data, target, test_size):
    ##было б все легко, если б не так все печально хаха данный встроенный метод работает только со списками, поэтому пришлось выкручиваться:)
    X_train, Y_train, X_test, Y_test = skmultilearn.model_selection.iterative_stratification.iterative_train_test_split(np.array(data), np.array(target), test_size = test_size)

    X_train = pd.DataFrame(X_train, columns =list(COL))
    X_test = pd.DataFrame(X_test, columns =list(COL))
    Y_train = pd.DataFrame(Y_train, columns = list(TARGET_COLS))
    Y_test = pd.DataFrame(Y_test, columns =list(TARGET_COLS))

    X_train = turning_types(X_train)
    X_test = turning_types(X_test)

    return X_train, Y_train, X_test, Y_test


In [394]:
X_train, Y_train, X_test, Y_test = split_data(train, target, test_size = 0.2)

In [420]:
##CATBOOST
model = CatBoostClassifier(
    iterations=1000, 
    loss_function='MultiLogloss', 
    eval_metric='MultiLogloss',
    learning_rate=0.01, 
    bootstrap_type='Bayesian', 
    boost_from_average=False,
    ctr_leaf_count_limit=3,   ## param for controling  cat_features
    leaf_estimation_iterations=1, 
    leaf_estimation_method='Gradient',
    cat_features = CATEGORIES_COL_AFTER_PREP)

model_cast = MultiOutputClassifier(model)

pipeline_castboost = Pipeline ([
    ('model_cast', model_cast)])

pipeline_castboost.fit(X_train,Y_train)

c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.6925516	total: 22.9ms	remaining: 22.9s
1:	learn: 0.6919989	total: 43.4ms	remaining: 21.6s
2:	learn: 0.6913742	total: 66.9ms	remaining: 22.2s
3:	learn: 0.6907322	total: 89.6ms	remaining: 22.3s
4:	learn: 0.6902015	total: 102ms	remaining: 20.3s
5:	learn: 0.6895236	total: 125ms	remaining: 20.6s
6:	learn: 0.6889215	total: 147ms	remaining: 20.9s
7:	learn: 0.6883143	total: 171ms	remaining: 21.2s
8:	learn: 0.6876543	total: 194ms	remaining: 21.3s
9:	learn: 0.6870651	total: 216ms	remaining: 21.4s
10:	learn: 0.6864694	total: 240ms	remaining: 21.6s
11:	learn: 0.6858672	total: 256ms	remaining: 21.1s
12:	learn: 0.6853293	total: 278ms	remaining: 21.1s
13:	learn: 0.6848058	total: 305ms	remaining: 21.5s
14:	learn: 0.6841693	total: 328ms	remaining: 21.6s
15:	learn: 0.6836013	total: 351ms	remaining: 21.6s
16:	learn: 0.6829993	total: 375ms	remaining: 21.7s
17:	learn: 0.6824255	total: 401ms	remaining: 21.9s
18:	learn: 0.6818272	total: 424ms	remaining: 21.9s
19:	learn: 0.6812847	total: 456ms	rem

In [477]:
y_predict = pipeline_castboost.predict(X_test)
y_predict_proba = pipeline_castboost.predict_proba(X_test)
metrics_print_for_catboost(y_predict, y_predict_proba, Y_test, 'micro')
metrics_print_for_catboost(y_predict, y_predict_proba, Y_test, 'samples')


micro
precision  0.6530612244897959
recall  0.4129032258064516
f1  0.5059288537549407
ROCAUC 0.8269506999391358


samples
precision  0.3368421052631579
recall  0.2399122807017544
f1  0.2668421052631579
ROCAUC [0.7364082  0.59409341 0.78365009 0.79901508 0.54233254]




c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dsfs-\AppData\Local\Pr

In [557]:
real_pipe = Pipeline([
    ('scaler', StandardScaler())])

cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore',sparse=False))
])

preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, REAL_COLS),
    ('cat_cols', cat_pipe, CATEGORIES_COL_AFTER_PREP),
    ('cat_bost_cols', ce.CountEncoder(), CATEGORIES_COL_AFTER_PREP), ##for work with cat_features
]
)

model_ltb = ltb.LGBMClassifier(
    boosting_type ='dart',
    num_leaves = 20,
    learning_rate = 0.01,
    n_estimators = 500
)

model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model_ltb)
]
)

pipline_ltb = MultiOutputClassifier(model_pipe, n_jobs=4)
pipline_ltb.fit(X_train, Y_train)

MultiOutputClassifier(estimator=Pipeline(steps=[('preprocess',
                                                 ColumnTransformer(transformers=[('real_cols',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['Возраст '
                                                                                   'курения',
                                                                                   'Сигарет '
                                                                                   'в '
                                                                                   'день',
                                                                                   'Возраст '
                                                                                   'алког']),
                                                                                 ('cat_cols',
                                                                                  Pipeline(steps=[('ohe',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse=False))]),
                                                                                  ['Пол',
                                                                                   'Семья',
                                                                                   'Этнос',
                                                                                   'Национальность',
                                                                                   'Религия'...
                                                                                   'кур',
                                                                                   'Алкоголь',
                                                                                   'Сова/Жаворонок/Голубь']),
                                                                                 ('cat_bost_cols',
                                                                                  CountEncoder(combine_min_nan_groups=True),
                                                                                  ['Пол',
                                                                                   'Семья',
                                                                                   'Этнос',
                                                                                   'Национальность',
                                                                                   'Религия',
                                                                                   'Профессия',
                                                                                   'Статус '
                                                                                   'Курения',
                                                                                   'Частота '
                                                                                   'пасс '
                                                                                   'кур',
                                                                                   'Алкоголь',
                                                                                   'Сова/Жаворонок/Голубь'])])),
                                                ('model',
                                                 LGBMClassifier(boosting_type='dart',
                                                                learning_rate=0.01,
                                          

In [558]:
y_predict = pipline_ltb.predict(X_test)
y_predict_proba = pipline_ltb.predict_proba(X_test)
metrics_print_for_lightgbm(y_predict, y_predict_proba, Y_test, 'micro')
metrics_print_for_lightgbm(y_predict, y_predict_proba, Y_test, 'samples')

micro
precision  0.5
recall  0.2064516129032258
f1  0.2922374429223744
ROCAUC 0.7444268614323393


samples
precision  0.16842105263157894
recall  0.11798245614035087
f1  0.1319298245614035
ROCAUC [0.56534091 0.48317308 0.47487633 0.53062481 0.56817409]




c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(tr

In [559]:
from joblib import dump, load
path_to_model = '../models/'
dump(pipeline_castboost, os.path.join(path_to_model, 'pipeline_castboost.joblib')) 
dump(pipline_ltb, os.path.join(path_to_model, 'pipline_ltb.joblib')) 


['../models/pipline_ltb.joblib']

## Inference

In [560]:
#processed_data_path = '../data/processed/'
#path_to_model = '../models/'
test = pd.read_pickle(os.path.join(processed_data_path, 'test.pkl'))
model_1  = load(os.path.join(path_to_model, 'pipeline_castboost.joblib')) 
model_2 = load(os.path.join(path_to_model, 'pipline_ltb.joblib')) 

In [562]:
y_pred_1  = model_1.predict(test)
print(y_pred_1)
y_pred_2  = model_2.predict(test)
print(y_pred_2)

c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [1 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]]
[[1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 ...
 [1 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]]
